In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

df = pd.read_csv("/content/car_dataset.csv")

df = df[df['Price'] != 'Ask For Price'].dropna()
df['Price'] = df['Price'].str.replace(',', '').astype(int)
df['kms_driven'] = df['kms_driven'].str.replace(' kms', '').str.replace(',', '')
df['kms_driven'] = df['kms_driven'].replace('', np.nan).dropna().astype(int)
df['year'] = df['year'].astype(int)

X = df[['company', 'year', 'kms_driven', 'fuel_type']]
y = df['Price']

categorical = ['company', 'fuel_type']
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical)
], remainder='passthrough')

model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=200, random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)

joblib.dump(model, 'car_price_model.pkl')

def predict_price(company, year, kms, fuel_type):
    new_data = pd.DataFrame([[company, year, kms, fuel_type]],
                            columns=['company', 'year', 'kms_driven', 'fuel_type'])
    return int(model.predict(new_data)[0])

car_name = input("Car Name : ").strip()
company = input("Company : ").strip()
try:
    year = int(input("Year of Purchase (e.g., 2018): ").strip())
    kms = int(input("Kilometers Driven (e.g., 25000): ").strip())
except:
    print("Please enter valid numbers for year and kilometers.")
    raise SystemExit
fuel_type = input("Fuel Type (Petrol/Diesel/CNG): ").strip()

try:
    result = predict_price(company, year, kms, fuel_type)
    print(f"\nPredicted Price for '{car_name}': ₹{result:,}")
except Exception as e:
    print("Prediction failed. Please check your input values.")


Car Name : Q3
Company : BMW
Year of Purchase (e.g., 2018): 2025
Kilometers Driven (e.g., 25000): 1000
Fuel Type (Petrol/Diesel/CNG): Petrol

Predicted Price for 'Q3': ₹640,319
